In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

from keras import Model
from keras.layers import Input, LSTM, Dense



In [11]:
seq_length = 100
step = seq_length
# maxlen = 1000

df = pd.read_csv("data.csv", index_col="index")
df = df.reindex(np.random.permutation(df.index))[:1000]
df.columns = ["label", "text"]
labels = df["label"].to_numpy()
texts = df["text"].to_numpy()

In [12]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(texts)
sequences = np.array(tokenizer.texts_to_sequences(texts))
# x = pad_sequences(x, maxlen=maxlen, padding="post")

label_encoder = LabelEncoder()
label_encoder.fit(["Rock", "Pop", "Hip Hop"])
encoded_labels = label_encoder.transform(labels)

x = []
y = []
for label_ind, sequence in enumerate(sequences):
    for i in range(0, len(sequence) - seq_length, step):
        x.append(sequence[i: i+seq_length])
        y.append(encoded_labels[label_ind])

x = to_categorical(x)
y = to_categorical(y)

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.4, random_state=42)

print(train_x.shape)
print(train_y.shape)


(8734, 100, 104)
(8734, 3)


In [17]:
n_lstm_layers = 3

input_layer = Input(x[0].shape)
z = input_layer
for i in range(n_lstm_layers):
    z = LSTM(64, return_sequences=i != n_lstm_layers-1)(z)
out = Dense(3, activation="softmax")(z)
model = Model(inputs=[input_layer], outputs=[out])
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()
    

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100, 104)]        0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 64)           43264     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total params: 109,507
Trainable params: 109,507
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10)

Epoch 1/10
273/273 [==============================] - 6s 22ms/step - loss: 1.0898 - accuracy: 0.3804 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
273/273 [==============================] - 5s 19ms/step - loss: 1.0606 - accuracy: 0.4414 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
273/273 [==============================] - 5s 19ms/step - loss: 1.0482 - accuracy: 0.4481 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
273/273 [==============================] - 5s 19ms/step - loss: 1.0426 - accuracy: 0.4515 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
273/273 [==============================] - 5s 20ms/step - loss: 1.0385 - accuracy: 0.4637 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
273/273 [==============================] - 6s 21ms/step - loss: 1.0338 - accuracy: 0.4557 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
273/273 [==============================] - 5s 20ms/step - loss: 1.0287 - accuracy: 